In [8]:
import time
import random
from typing import Dict
import torch



# Instruction Finetuning LLMs with QLoRA for RAG

Large Language Models are typically trained as models that simply predict the next word in a sequence.  While this leads to very powerful machines, they don't typically come equipped to deal with certain behaviors, such as following instructions.  In this lab, we will demonstrate how to fine tune a base Large Language Model to better respond to instructions with context, which is a requirement for RAG.  By fine tuning the model in this way, we can teach it to stop better, hallucinate less, and generally behave in a more desirable way.

## **Important Note**

***We are finetuning a base model for RAG for instructional purposes on how finetuning can change the behavior of models. In practice, many models provide instruction fine-tuned models which will give better results than we can produce here for RAG because they are trained on many more data examples. For example (mistralai/Mistral-7B-v0.1 vs mistralai/Mistral-7B-Instruct-v0.1) and (meta-llama/Llama-2-7b-hf vs. meta-llama/Llama-2-7b-chat-hf). Try and get the best performance out of the finetuning but don't expect it to work perfectly..***

- [Preparing the Dataset](#preparing-the-dataset)
- [Selecting the Base Pre-trained Model](#selecting-the-base-pre-trained-model)
- [Finetuning the Model](#finetuning-the-model)

## Preparing the Dataset

Fine-tuning LLMs is primarily used for teaching the model new behavior, such as better responding to instructions, responding with certain tones, or acting more as a conversational chatbot.  

The dataset for finetuning LLMs are text entries formatted in the way ***THAT WE WISH FOR AN INTERACTION WITH THE MODEL TO LOOK LIKE***.  For example, if we wish for the model to follow instructions better with context, we should provide a dataset which gives examples of it following instructions provided with context.  **This is almost exactly like few-shot prompting, but reinforcing the behavior even further by actually modifying some of the weights of the model.**

A few tips from ChatGPT:

Generative Dataset:

    1. Include a dataset of input queries or prompts along with human-generated responses. This is your generative dataset.

    2. Make sure that the responses are diverse, well-written, and contextually appropriate for the given queries.

    3. It's important to have a variety of responses to encourage the model to generate creative and contextually relevant answers.

Training Data Quality:

    1. Ensure that your training dataset is of high quality and accurately represents the task you are fine-tuning for.

    2. Remove any instances that contain incorrect or misleading information.

    3. Filter out instances in your training data where the model is likely to hallucinate or generate incorrect information.

    4. Manually review and filter out examples that may lead to misinformation.

    5. Use data augmentation techniques to artificially increase the diversity of your dataset. However, be cautious with augmentation to ensure that the generated samples remain contextually relevant and accurate.
```

### Dataset using `datasets`

The dataset that we will be using for instruction fine-tuning is a dataset hand-curated by databricks for instruction following called "dolly-15k".

In [9]:
from datasets import load_dataset, Dataset
import pandas as pd

def load_modified_dataset():
    dataset = load_dataset("databricks/databricks-dolly-15k", split = "train")
    df = dataset.to_pandas()
    df['keep'] = True
    
    # Keep entries with correct answer as well
    df = df[(df['category'].isin(['closed_qa', 'information_extraction', 'open_qa'])) & df['keep']]
    
    return Dataset.from_pandas(
        df[['instruction', 'context', 'response']], 
        preserve_index = False)
    
dataset = load_modified_dataset()
# dataset = dataset.select(range(200))

The base dataset contains columns for an `instruction`, an optional `context`, and a `response` that we want the bot to respond to.  However, to feed it into the model for finetuning, we need to combine each column so that 1 sample corresponds to 1 example interaction with the model.  

This 1 sample should be an example to the LLM about:

1. How we wish to interact with the model (prompt)
2. How we want the model to respond

Remember, these generative LLMs are trained to read in a provided prompt, and essentially auto-complete the text!

In [10]:
def format_instruction(sample : Dict) -> str:
    """Combine a row to a single str"""
    return f"""### Context:
{sample['context']}

### Question:
Using only the context above, {sample['instruction']}

### Response:
{sample['response']}
"""

We will provide this as the entire prompt to the model for training, using the Causal Language Modeling objective for loss.

```
### Context:
{context}

### Question:
Using only the context above, {instruction}

### Response:
{response}
```

## Selecting the Base Pre-trained Model

Once we have the data, we can select the base model that we would like to fine tune for this behavior.  

The model that we will select is the `mistralai/mistral-7b` base model.  This is a 7.3b parameter model, quite small in the grand scheme of LLMs, but one that produces good quality results, especially compared to many other open source models.

### Quantization using `bitsandbytes`

LLMs are extremely memory intensive.  One trick that is commonly used when working with LLMs to reduce memory usage as well as increase computational speed for both inference and training, is reducing the precision of the weights from full precision 32-bit floating points (fp32) to lower precisions such as int8, fp4, nf4, etc.  This is known as quantization.  Research has shown that quantization often times has minimal impact on the quality of generations, but this is on a case-by-case basis. 

In this example, we will be quantizing and fine-tuning using normal-float 4 bit (nf4).  In practice, the quantization behind the scenes is handled by the `bitsandbytes` library.

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM
# Hugging Face Base Model ID
model_id = "mistralai/Mistral-7B-v0.1"
is_peft = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

if is_peft:
    # load base LLM model with PEFT Adapter
    model = AutoPeftModelForCausalLM.from_pretrained(
        model_id,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        use_flash_attention_2=True,
        quantization_config = bnb_config
    )
else:
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        quantization_config = bnb_config,
        use_flash_attention_2=True
    )

model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

With the model loaded up, we are ready to finetune using our dataset.

## Finetuning the Model

There are two main ways to finetune a large language model:

1. Pre-training/Full Finetuning

    In this situation, all of the model weights (all 7b of them) are set to be trainable and tweaked during training.  This can lead to the most dramatic changes in model behavior but is also the most computationally expensive.  
    
    When initially training the model, also known as pre-training, this is necessarily done and where you see the extreme computational costs show up (i.e. 500 A100 80GB GPUs trained for 10000 hours, etc...).

2. Parameter Efficient Fine-Tuning (PEFT)

    Parameter efficient finetuning methods are an alternative to full finetuning where, instead of training the parameters of the pre-trained model, a subset of new parameters are trained without touching the base model weights. These new trainable parameters are injected into the model mathematically at different points to change the outcome.  There are a handful of methods that use this approach such as Prompt Tuning, P-Tuning, and Low-Rank Adaptation (LoRA).  For this lab, we will focus on LoRA.  

    LoRA methods introduce a set of trainable rank-decomposition matrices (update matrices) which can be used to modify the existing weights of the pre-trained model.  The typical location that these matrices are placed are within the attention layers, so they are not exclusive to LLMs.  The size of these update matrices can be controlled by  setting the desired rank of the matrix (lora_r), with smaller rank corresponding to smaller matrices and thus fewer trainable parameters.   During fine-tuning, only these update matrices are tuned and often times, this makes the total number of trainable parameters a very small fraction of the total number of weights.

### Finetuning using `peft`

To configure the model for paremeter efficient fine-tuning and LoRA, we will use the `peft` package.  Specifically, we will define our Lora parameters and also set to the taks to `CAUSAL_LM` to train the model for generative purposes.  Because we also quantized the model to 4-bit, we will also be using a state-of-the-art method called Quantized LoRA (QLoRA) to do this training in low precision to save memory.


In [12]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

if is_peft:
    model = prepare_model_for_kbit_training(model)
    model._mark_only_adapters_as_trainable()
else:
    # LoRA config for QLoRA
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=['v_proj', 'down_proj', 'up_proj', 'o_proj', 'q_proj', 'gate_proj', 'k_proj']
    )

    # prepare model for training with low-precision
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)

### Running the trainer with `trl`

Now that we have prepared the data, loaded the model in 4-bit, and configured our LoRA finetuning according to our model, we are ready to train the model. Training of LLMs for generative purposes uses the causal language modeling objective.  Briefly, this specifies that when calculating attention, the model should only be able to consider things "to the left".  So for a sentence, it can only decide what to generate by looking at all of the words that came before it.  

A very useful wrapper for training transformer based models is the Supervised Fine-Tuning Trainer (`SFTrainer`) provided by the `trl` library.  While the supervised fine tuning is typically used in the context of reinforcement learning, for our purposes, it simply refers to providing the model with examples of input, and response.  All of the actual training, including computing gradients, tweaking the optimizer, batching the data, evaluation will be done behind the scenes using the `SFTrainer` wrapper.  This will conduct the finetuning that we want after we pass in the dataset and hyperparameters.  This is much more efficient and robust than writing our own training code.

In [13]:
from transformers import TrainingArguments
from trl import SFTTrainer

args = TrainingArguments(
    output_dir="./mistral-7b-int4-dolly", 
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=1, # batch size per batch
    gradient_accumulation_steps=10, # effective batch size
    gradient_checkpointing=True, 
    gradient_checkpointing_kwargs={'use_reentrant':True},
    optim="paged_adamw_32bit",
    logging_steps=1, # log the training error every 10 steps
    save_strategy="steps",
    save_total_limit = 2, # save 2 total checkpoints
    ignore_data_skip=True,
    save_steps=2, # save a checkpoint every 1 steps
    learning_rate=1e-3,
    bf16=True,
    tf32=True,
    max_grad_norm=1.0,
    warmup_steps=100,
    lr_scheduler_type="constant",
    disable_tqdm=True
)

# https://huggingface.co/docs/trl/sft_trainer#packing-dataset--constantlengthdataset-
# max seq length for packing
max_seq_length = 2048 
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    packing=True,
    formatting_func=format_instruction, # our formatting function which takes a dataset row and maps it to str
    args=args,
)

With all of the configuration done, we can now run our training.  On an A10g, this takes about 1 hours to run, after which it will save the LoRA weights to the `mistral-7b-int4-dolly` directory.

In [14]:
start = time.time()
trainer.train(resume_from_checkpoint=False) # reduce batch size
trainer.save_model()
end = time.time()
print(f"{end - start}s")

/home/kevinjesse/AccentureTeach/genai-bootcamp-curriculum/genai-bootcamp-curriculum/1-daily-exercises/synthetic-data-exercise/venv/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/kevinjesse/AccentureTeach/genai-bootcamp-curriculum/genai-bootcamp-curriculum/1-daily-exercises/synthetic-data-exercise/venv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.5285, 'grad_norm': 1.276779294013977, 'learning_rate': 0.001, 'epoch': 0.036101083032490974}
{'loss': 1.49, 'grad_norm': 2.744922399520874, 'learning_rate': 0.001, 'epoch': 0.07220216606498195}


/home/kevinjesse/AccentureTeach/genai-bootcamp-curriculum/genai-bootcamp-curriculum/1-daily-exercises/synthetic-data-exercise/venv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.4345, 'grad_norm': 1.558997392654419, 'learning_rate': 0.001, 'epoch': 0.10830324909747292}


KeyboardInterrupt: 

After the model has finished training, it is ready to be used.  Now, hopefully, when the model sees the prompt that we crafted before, it will know how to respond.